<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/qiskit_kernel_sweep_full_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit scikit-learn numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
qiskit_kernel_sweep_full.py

End-to-end script to:
  - Generate 2D synthetic data with specified class imbalances
  - Build ZZ and Pauli quantum feature maps at various depths
  - Compute fidelity² kernels
  - Center kernels and compute full & mini-batch kernel–target alignments
  - Train SVMs (unweighted vs. class-weighted) via precomputed kernels
  - Plot fidelity² histograms for same- vs. cross-class pairs
"""

import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from qiskit.circuit.library import zz_feature_map, PauliFeatureMap
from qiskit.quantum_info import Statevector, state_fidelity

warnings.filterwarnings("ignore", category=DeprecationWarning)


def generate_data(n_samples, imbalance, random_state=None):
    if imbalance >= 1.0:
        weights = [0.5, 0.5]
    else:
        weights = [imbalance, 1.0 - imbalance]
    X, y = make_classification(
        n_samples=n_samples,
        n_features=2,
        n_redundant=0,
        n_clusters_per_class=1,
        weights=weights,
        class_sep=2.0,
        random_state=random_state
    )
    return X, y


def build_feature_map(name, reps):
    if name == "ZZ":
        return zz_feature_map(
            feature_dimension=2,
            reps=reps,
            entanglement='circular',
            insert_barriers=False
        )
    elif name == "Pauli":
        return PauliFeatureMap(
            feature_dimension=2,
            reps=reps,
            paulis=['X', 'Y', 'Z'],
            entanglement='circular',
            insert_barriers=False
        )
    else:
        raise ValueError(f"Unknown feature map: {name}")


def compute_fidelity_kernel(fmap, X):
    n = len(X)
    # Prepare statevectors for each input sample
    states = [Statevector.from_instruction(fmap.assign_parameters(x)) for x in X]
    K = np.zeros((n, n), float)
    for i in range(n):
        for j in range(i, n):
            f2 = state_fidelity(states[i], states[j]) ** 2
            K[i, j] = K[j, i] = f2
    return K


def center_kernel(K):
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    return K - one_n @ K - K @ one_n + one_n @ K @ one_n


def kernel_target_alignment(Kc, y):
    yy = np.outer(y, y)
    num = np.trace(Kc @ yy)
    den = np.linalg.norm(Kc, 'fro') * np.linalg.norm(yy, 'fro')
    return num / den if den > 0 else 0.0


def mini_batch_alignment(K, y, batch_size=25, n_batches=100, random_state=None):
    rng = np.random.default_rng(random_state)
    y_signed = 2 * y - 1
    aligns = []
    n = len(y)
    for _ in range(n_batches):
        idx = rng.choice(n, size=min(batch_size, n), replace=False)
        Kb = K[np.ix_(idx, idx)]
        yb = y_signed[idx]
        yyb = np.outer(yb, yb)
        num = np.trace(Kb @ yyb)
        den = np.linalg.norm(Kb, 'fro') * np.linalg.norm(yyb, 'fro')
        aligns.append(num / den if den > 0 else 0.0)
    return np.mean(aligns)


def svm_cv(K, y, weighted=False, max_splits=5):
    classes, counts = np.unique(y, return_counts=True)
    n_splits = min(max_splits, counts.min())
    if len(classes) < 2 or n_splits < 2:
        return np.array([])
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    svc = SVC(kernel='precomputed', class_weight='balanced' if weighted else None)
    return cross_val_score(svc, K, y, cv=cv)


def plot_fidelity_histograms(K, y, title=None, bins=30):
    idx = np.triu_indices_from(K, k=1)
    vals = K[idx]
    yy = np.outer(y, y)
    same = vals[yy[idx] == 1]
    cross = vals[yy[idx] != 1]
    plt.figure(figsize=(6, 4))
    plt.hist(same, bins=bins, alpha=0.6, label='same-class')
    plt.hist(cross, bins=bins, alpha=0.6, label='cross-class')
    plt.xlabel('Fidelity²')
    plt.ylabel('Count')
    if title:
        plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()


def main():
    np.random.seed(42)
    n_samples = 50
    imbalance_levels = [1.0, 0.75, 0.5, 0.2, 0.1]
    feature_maps = ["ZZ", "Pauli"]
    depths = [1, 3, 5]

    for imb in imbalance_levels:
        X, y = generate_data(n_samples, imb, random_state=1)
        print(f"\n=== Imbalance {int(imb * 100)}% (N={n_samples}) ===")
        for name in feature_maps:
            for d in depths:
                fmap = build_feature_map(name, d)
                K = compute_fidelity_kernel(fmap, X)
                Kc = center_kernel(K)
                full_al = kernel_target_alignment(Kc, y)
                mb_al = mini_batch_alignment(
                    K, y, batch_size=25, n_batches=100, random_state=0
                )
                scores = svm_cv(K, y, weighted=False)
                scores_b = svm_cv(K, y, weighted=True)
                fmt = lambda s: f"{s.mean():.3f}±{s.std():.3f}" if s.size else "n/a"
                print(
                    f"{name:6s} depth={d} | full_align={full_al:.3f} "
                    f"| mb_align={mb_al:.3f} | SVM={fmt(scores)} "
                    f"| SVM_bal={fmt(scores_b)}"
                )
                plot_fidelity_histograms(
                    K,
                    y,
                    title=f"{name} depth={d}, Imb={int(imb * 100)}%"
                )


if __name__ == "__main__":
    main()